In [53]:
import os

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [54]:
data = pd.read_csv(os.path.join("data", "census_cleaned.csv"))

In [55]:
data.drop(columns=["Unnamed: 0"], inplace=True)
data.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [56]:
train, test = train_test_split(data, test_size=0.20)

cat_features = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
]
num_features = [
    "age", "education-num", "capital-gain", "capital-loss", "hours-per-week"
]

In [57]:
train.shape

(26048, 15)

In [69]:
encoder = OneHotEncoder().fit(train[cat_features])

In [72]:
encoder

OneHotEncoder()

In [70]:
one_hot_encoded_cat_vars = encoder.transform(train[cat_features])

In [71]:
one_hot_encoded_cat_vars.A.shape

(26048, 102)

In [60]:
train[num_features].to_numpy().shape

(26048, 5)

In [61]:
X_train_array = np.concatenate([one_hot_encoded_cat_vars.A,train[num_features].to_numpy()], axis=1)

In [64]:
train["salary"]

13566    <=50K
20050    <=50K
14268    <=50K
1737     <=50K
10167    <=50K
         ...  
10058    <=50K
12086     >50K
4403     <=50K
10230     >50K
6989     <=50K
Name: salary, Length: 26048, dtype: object

In [68]:
(train["salary"] == ">50K").to_numpy().reshape(-1, 1)

array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])